# Import libraries

In [2]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Load data

In [3]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "is_healthy.csv"), index_col=[0, 1])
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-11  SAMEA3136645        0.000000                     0.000000   
         SAMEA3136644        0.015974                     0.015974   
         SAMEA3136672        0.000000                     0.000000   
         SAMEA3136637        0.000000                     0.000000   
         SAMEA3136639        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-11  SAMEA3136645                                            0.000000   
         SAMEA3136644                                            0.015974   
         SAMEA3136672                                            0.000000   
         SAMEA3136637                                            0.000000   
         SAMEA3136639                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-11  SAMEA3136645                                               0.000000                      
         SAMEA3136644                                               0.015974                      
         SAMEA3136672                                               0.000000                      
         SAMEA3136637                                               0.000000                      
         SAMEA3136639                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-11  SAMEA3136645                                               0.000000                                             
         SAMEA3136644                                               0.015974                                             
         SAMEA3136672                                               0.000000                                             
         SAMEA3136637                                               0.000000                                             
         SAMEA3136639                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-11  SAMEA3136645                                               0.000000                                                                   
         SAMEA3136644                                               0.015974                                                                   
         SAMEA3136672                                               0.000000                                                                   
         SAMEA3136637                                               0.000000                                                                   
         SAMEA3136639                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-11  SAMEA3136645                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-11  SAMEA3136645                              True
         SAMEA3136644                              True
         SAMEA3136672                              True
         SAMEA3136637                              True
         SAMEA3136639                              True

# Iterate through subset sizes and compare ISV and 10-fold

In [19]:
# indentify groups to partition dataset

sample_studies = np.array(X.index.get_level_values(0))
o = OrdinalEncoder()
groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
logo = LeaveOneGroupOut()

In [20]:
studies = np.unique(X.index.get_level_values(0))
studies

array(['GMHI-11', 'GMHI-13', 'GMHI-14', 'GMHI-15', 'GMHI-17', 'GMHI-2',
       'GMHI-21', 'GMHI-23', 'GMHI-24', 'GMHI-26', 'GMHI-27', 'GMHI-28',
       'GMHI-3', 'GMHI-31', 'GMHI-32', 'GMHI-33', 'GMHI-4', 'GMHI-5',
       'GMHI-9', 'GMHI-V-35', 'GMHI-V-36', 'GMHI-V-38', 'GMHI-V-40',
       'GMHI-V-41', 'P11', 'P110', 'P121', 'P132', 'P135', 'P140', 'P15',
       'P17', 'P2', 'P21', 'P24', 'P32', 'P34', 'P39', 'P48', 'P53',
       'P56', 'P57', 'P69', 'P74', 'P77', 'P8', 'P80', 'P81', 'P87',
       'P88', 'P89', 'P9', 'P98'], dtype=object)

In [25]:
info = []

In [26]:
num_iterations = 5

for subset_size in range(30, studies.shape[0] + 1):
    print(subset_size)
    for i in range(num_iterations):
        subset = np.random.choice(studies, size=subset_size, replace=False)
        X_curr = X.loc[subset]
        y_curr = y.loc[subset]
        
        # setup ISV stuff
        curr_sample_studies = np.array(X_curr.index.get_level_values(0))
        o = OrdinalEncoder()
        groups = o.fit_transform(curr_sample_studies.reshape((len(curr_sample_studies), 1))).flatten()
        logo = LeaveOneGroupOut()
        
        # setup kfold stuff
        kfold = KFold(n_splits=10, shuffle=True)
        
        # instantiate the logistic regression classifier (gmhi2)
        gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")
        
        # generate logit predictions in 10 fold cross validation
        GMHI2_scores_10fold = cross_val_predict(gmhi2, X_curr > config.PRESENCE_CUTOFF, y_curr.values.flatten(), method="decision_function",
            cv=kfold, verbose=0, n_jobs=-1
        )
        
        # instantiate the logistic regression classifier (gmhi2)
        gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")
        
        GMHI2_scores_ISV = cross_val_predict(gmhi2, X_curr > config.PRESENCE_CUTOFF, y_curr.values.flatten(), method="decision_function",
            cv=logo, groups=groups, verbose=0, n_jobs=-1
        )
        
        # evaluate performance
        kfold_perf = balanced_accuracy_score(GMHI2_scores_10fold > 0, y_curr)
        isv_perf = balanced_accuracy_score(GMHI2_scores_ISV > 0, y_curr)
        
        info.append({
            "type" : "kfold",
            "bal" : kfold_perf,
            "i" : i,
            "size" : subset_size
        })
        
        info.append({
            "type" : "isf",
            "bal" : isv_perf,
            "i" : i,
            "size" : subset_size
        })
        

30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [24]:
info

NameError: name 'info' is not defined